In [4]:
!pip install -r requirements.txt
import argparse
import torch
import torch.nn as nn
from evaluate import *
from utils import *
from tqdm.notebook import tqdm
from data_loader import *
from stability_loss_function import *

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/RobustBench/robustbench.git to /tmp/pip-req-build-cdsd2hhb
  Running command git clone --filter=blob:none --quiet https://github.com/RobustBench/robustbench.git /tmp/pip-req-build-cdsd2hhb
  Resolved https://github.com/RobustBench/robustbench.git to commit 776bc95bb4167827fb102a32ac5aea62e46cfaab
  Preparing metadata (setup.py) ... done
  Using cached autoattack-0.1-py3-none-any.whl


In [7]:
parser = argparse.ArgumentParser(description="Hyperparameters for the script")

# Define the hyperparameters controlled via CLI 'Ding2020MMA'

parser.add_argument('--in_dataset', type=str, default='cifar100', choices=['cifar10', 'cifar100'], help='The in-distribution dataset to be used')
parser.add_argument('--threat_model', type=str, default='Linf', help='Adversarial threat model for robust training')
parser.add_argument('--noise_std', type=float, default=1, help='Standard deviation of noise for generating noisy fake embeddings')
parser.add_argument('--attack_eps', type=float, default=8/255, help='Perturbation bound (epsilon) for PGD attack')
parser.add_argument('--attack_steps', type=int, default=10, help='Number of steps for the PGD attack')
parser.add_argument('--attack_alpha', type=float, default=2.5 * (8/255) / 10, help='Step size (alpha) for each PGD attack iteration')

args = parser.parse_args('')

# Set the default model name based on the selected dataset
if args.in_dataset == 'cifar10':
    default_model_name = 'Rebuffi2021Fixing_70_16_cutmix_extra'
elif args.in_dataset == 'cifar100':
    default_model_name = 'Wang2023Better_WRN-70-16'

parser.add_argument('--model_name', type=str, default=default_model_name, choices=['Rebuffi2021Fixing_70_16_cutmix_extra', 'Wang2023Better_WRN-70-16'], help='The pre-trained model to be used for feature extraction')

# Re-parse arguments to include model_name selection based on the dataset
args = parser.parse_args('')
num_classes = 10 if args.in_dataset == 'cifar10' else 100

trainloader, testloader,test_set, ID_OOD_loader = get_loaders(in_dataset=args.in_dataset)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [6]:
robust_backbone = load_model(model_name=args.model_name, dataset=args.in_dataset, threat_model=args.threat_model).to(device)


attack_eps = 8/255
attack_steps = 10
attack_alpha = 2.5 * attack_eps / attack_steps

test_attack = PGD_MSP(robust_backbone, eps=attack_eps, steps=attack_steps, alpha=attack_alpha, num_classes=num_classes)


auc_MSP(robust_backbone, ID_OOD_loader , device, num_classes)
adv_auc = auc_MSP_adversarial(model=robust_backbone,  test_loader=ID_OOD_loader, test_attack=test_attack, device=device, num_classes=num_classes)
#CIFAR-10 vs. CIFAR100

  0%|          | 0/1250 [00:00<?, ?batch/s]

0.8422808549999998


  0%|          | 0/1250 [00:00<?, ?batch/s]

0.434837885


In [ ]:
robust_backbone = load_model(model_name=args.model_name, dataset=args.in_dataset, threat_model=args.threat_model).to(device)


attack_eps = 8/255
attack_steps = 10
attack_alpha = 2.5 * attack_eps / attack_steps

test_attack = PGD_MSP(robust_backbone, eps=attack_eps, steps=attack_steps, alpha=attack_alpha, num_classes=num_classes)


auc_MSP(robust_backbone, ID_OOD_loader , device, num_classes)
adv_auc = auc_MSP_adversarial(model=robust_backbone,  test_loader=ID_OOD_loader, test_attack=test_attack, device=device, num_classes=num_classes)
#CIFAR-100 vs. CIFAR10

/home/hossein/.local/lib/python3.10/site-packages/robustbench/utils.py:165: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location=to

  0%|          | 0/1250 [00:00<?, ?batch/s]

0.7425861699999999


  0%|          | 0/1250 [00:00<?, ?batch/s]